In [18]:
# CREATE TABLE items (id bigserial PRIMARY KEY, content TEXT, embedding vector(1536));

In [19]:
import os
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
import openai
import psycopg2

In [20]:
load_dotenv(find_dotenv())

True

In [21]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Set up your OpenAI API key
# openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
print(OPENAI_API_KEY[:12])

sk-proj-vbza


In [22]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="host.docker.internal",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [23]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(input=prompt, model="text-embedding-ada-002")

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return "[" + ",".join(map(str, embedding)) + "]"

Second, find the most relevant movies for a provided user prompt by calculating the cosine distance (`<=>`) between the prompt's and movies' embeddings:


In [26]:
try:

    query = "Potentiodynamic polarization"
    query = " better time"
    query = "planning and designing"
    # Perform a cosine similarity search
    sql = f"""
       SELECT file_id, element_text FROM tbl_doc_elements WHERE element_text @@ websearch_to_tsquery('english','{query}');
        """
    cur.execute(sql)

    print("Query:", query)

    print("Most similar sentences:")

    for row in cur.fetchall():

        print(row)

except Exception as e:

    print("Error executing query", str(e))

Query: planning and designing
Most similar sentences:
('3712536d-0691-485d-92bc-857a4374a7f9', 'Discover the blueprint for planning, designing, and building a search experience that meets your users’ needs, your team’s resources, and (of course) your budget. With an overview of the latest tech and real-world case studies that highlight what’s possible, you can envision your future search experience and bring it to life.')


In [25]:
cur.close()
conn.close()